This is an example of a simple CNN developed, trained and utilized

AI was used to help generate the codebase

Note: Make sure that the tensorflow package is installed in your device.

In [20]:
# Lib imports
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, regularizers
import numpy as np

In [21]:
# DATASET DIRECTORY CONFIGURATION
# Download and unzip the dataset from Kaggle, set the directory paths accordingly.
train_dir = "train"  # e.g. './muffin-vs-chihuahua/train'
test_dir = "test"    # e.g. './muffin-vs-chihuahua/test'

In [22]:
# IMAGE PARAMETERS
# Used to resize the input images, also will determine the input size of your input layer.
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

In [23]:
# DATA PREPROCESSING & AUGMENTATION
# Optional but recommended for image processing tasks, especially with limited data.
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 3788 images belonging to 2 classes.
Found 945 images belonging to 2 classes.
Found 945 images belonging to 2 classes.
Found 1184 images belonging to 2 classes.
Found 1184 images belonging to 2 classes.


In [24]:
# SIMPLE CNN MODEL ARCHITECTURE

# Some modifications are applied
initial_learning_rate = 0.001
# We are combining ExponentialDecay with Adam optimizer for better learning rate management
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True
)

# Create the optimizer with the learning rate schedule
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# Applied dropout layers to reduce overfitting AND L2 regularization
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', 
                  kernel_regularizer=regularizers.l2(0.001),
                  input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),
    
    layers.Conv2D(64, (3, 3), activation='relu',
                  kernel_regularizer=regularizers.l2(0.001)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),
    
    layers.Conv2D(128, (3, 3), activation='relu',
                  kernel_regularizer=regularizers.l2(0.001)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),
    
    layers.Flatten(),
    layers.Dense(128, activation='relu',
                 kernel_regularizer=regularizers.l2(0.001)),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

In [25]:
# Configure the model optimizers, loss function, and metrics
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) # old
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [26]:
# TRAINING THE CNN
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

Epoch 1/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 51s 416ms/step - accuracy: 0.6513 - loss: 0.8658 - val_accuracy: 0.8063 - val_loss: 0.6287
Epoch 2/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 51s 416ms/step - accuracy: 0.6513 - loss: 0.8658 - val_accuracy: 0.8063 - val_loss: 0.6287
Epoch 2/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 58s 490ms/step - accuracy: 0.7624 - loss: 0.6190 - val_accuracy: 0.7915 - val_loss: 0.5355
Epoch 3/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 58s 490ms/step - accuracy: 0.7624 - loss: 0.6190 - val_accuracy: 0.7915 - val_loss: 0.5355
Epoch 3/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 54s 453ms/step - accuracy: 0.7946 - loss: 0.5261 - val_accuracy: 0.7810 - val_loss: 0.5790
Epoch 4/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 54s 453ms/step - accuracy: 0.7946 - loss: 0.5261 - val_accuracy: 0.7810 - val_loss: 0.5790
Epoch 4/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 51s 432ms/step - accuracy: 0.8173 - loss: 0.4850 - val_accuracy: 0.8402 - val_loss: 0.4549
Epoch 5/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 51s 432ms/step - accuracy: 0.8173 - loss: 0

In [27]:
# EVALUATE THE MODEL
test_loss, test_acc = model.evaluate(test_generator)
print(f"\n=== MODEL EVALUATION RESULTS ===")
print(f"Test Accuracy: {test_acc:.4f}")
print(f"Test Loss: {test_loss:.4f}")

37/37 ━━━━━━━━━━━━━━━━━━━━ 7s 194ms/step - accuracy: 0.8666 - loss: 0.4504
37/37 ━━━━━━━━━━━━━━━━━━━━ 7s 194ms/step - accuracy: 0.8666 - loss: 0.4504

=== MODEL EVALUATION RESULTS ===
Test Accuracy: 0.8666
Test Loss: 0.4504

=== MODEL EVALUATION RESULTS ===
Test Accuracy: 0.8666
Test Loss: 0.4504


In [ ]:
# SAVE THE MODEL
model.save('exercise_6_trained_model_improved.h5')
print("Model saved as: exercise_6_trained_model_improved.h5")

In [ ]:
# SIMPLE INFERENCE SCRIPT
from tensorflow.keras.preprocessing import image

def predict_image(img_path, model_path='exercise_6_trained_model_improved.h5'):
    model = tf.keras.models.load_model(model_path)
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    pred = model.predict(img_array)[0,0]
    label = "Chihuahua" if pred >= 0.5 else "Muffin"
    print(f"Prediction: {label} (confidence: {pred:.2f})")


In [30]:
# Example usage:
predict_image("test/muffin/img_0_0.jpg")
predict_image("test/chihuahua/img_0_1071.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step


Prediction: Muffin (confidence: 0.42)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
Prediction: Muffin (confidence: 0.22)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
Prediction: Muffin (confidence: 0.22)


## Question Answers

**a. Accuracy:** Run cell 9 above to see the test accuracy after training with dropout and regularization.

**b. Run 1 and Run 2 predictions:** Place your `run_1.jpg` and `run_2.jpg` images in the notebook directory, then run the cell below.

In [32]:
# PREDICTIONS FOR RUN 1 AND RUN 2
print("\n=== CUSTOM IMAGE PREDICTIONS ===")
print("\nRun 1 (using test/muffin/img_0_0.jpg):")
predict_image("test/muffin/img_0_0.jpg")
print("\nRun 2 (using test/chihuahua/img_0_1071.jpg):")
predict_image("test/chihuahua/img_0_1071.jpg")


=== CUSTOM IMAGE PREDICTIONS ===

Run 1 (using test/muffin/img_0_0.jpg):
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step


Prediction: Muffin (confidence: 0.42)

Run 2 (using test/chihuahua/img_0_1071.jpg):
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
Prediction: Muffin (confidence: 0.22)
